# Road Following - Live demo

## Load Trained Model

best_steering_model_xy.pth 파일을 통해서 젯봇이 자율주행한다.

In [ ]:
import torchvision
import torch


model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

In [ ]:
model.load_state_dict(torch.load('best_steering_model_xy.pth'))

In [ ]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

## Creating the Pre-Processing Function

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [ ]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

In [ ]:
from jetbot import Robot

robot = Robot()

In [ ]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')
ㅁ
display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

위 코드를 통해서 젯봇의 속도와 바퀴가 돌아가는 정도 등을 조정할 수 있다.

In [ ]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

In [ ]:
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
execute({'new': camera.value})

## Alcohol

젯봇과 연결된 포트 네임을 자신의 것에 맞게 변경해야 한다.

In [1]:
#alcohol code
import serial
import time


#set a port number & baud rate
PORT = '/dev/ttyACM0' #젯봇과 연결된 포트 네임
Baud = 9600

ARD = serial.Serial(PORT,Baud)

if not ARD.isOpen():
       ARD.open()
print('port is open', ARD.isOpen())

SerialException: [Errno 2] could not open port /dev/ttyACM0: [Errno 2] No such file or directory: '/dev/ttyACM0'

포트가 잘 연결되었다는 의미로 'port is open'가 출력된다.
아래의 코드를 실행하면 

In [ ]:
time.sleep(3)
def Decode(A):
    A = A.decode()
    A = str(A)
    if A[0]=='A':
        if(len(A)==11):
            Ard1=int(A[1:5])
            Ard2=int(A[5:9])
            result= Ard2
            return result
        else :
            print("Error_lack of number _ %d" %len(A))
            return False
    else :
        print ("Error_Wrong Signal")
        return False
    
def Ardread(): #return list [Ard1, Ard2]
    if ARD.readable():
        LINE = ARD.readline()
        code=Decode(LINE)
#        print(code)
        return code
    else:
        print("읽기 실패 from _Ardread_")
        
while (True):
    if Ardread()>160:
        print("Alcohol")
        robot.stop()
#           else:
        print("Run")
        camera.observe(execute, names='value')

In [ ]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()